Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [17]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [18]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
#   # Predictions for the training, validation, and test data.
prediction = tf.nn.softmax(logits)
#   valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
#   test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.705979
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.456081
Minibatch accuracy: 56.2%
Validation accuracy: 56.1%
Minibatch loss at step 100: 0.939831
Minibatch accuracy: 56.2%
Validation accuracy: 71.1%
Minibatch loss at step 150: 0.789652
Minibatch accuracy: 68.8%
Validation accuracy: 74.0%
Minibatch loss at step 200: 0.621068
Minibatch accuracy: 87.5%
Validation accuracy: 76.6%
Minibatch loss at step 250: 0.914265
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 300: 0.294531
Minibatch accuracy: 93.8%
Validation accuracy: 79.1%
Minibatch loss at step 350: 0.427471
Minibatch accuracy: 87.5%
Validation accuracy: 79.7%
Minibatch loss at step 400: 0.677672
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 450: 0.465037
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 500: 0.793101
Minibatch accuracy: 75.0%
Validation accuracy: 80.1%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.567309
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.841329
Minibatch accuracy: 37.5%
Validation accuracy: 43.5%
Minibatch loss at step 100: 1.109720
Minibatch accuracy: 68.8%
Validation accuracy: 68.8%
Minibatch loss at step 150: 0.713345
Minibatch accuracy: 75.0%
Validation accuracy: 72.0%
Minibatch loss at step 200: 0.555229
Minibatch accuracy: 81.2%
Validation accuracy: 76.9%
Minibatch loss at step 250: 1.079049
Minibatch accuracy: 68.8%
Validation accuracy: 75.5%
Minibatch loss at step 300: 0.283517
Minibatch accuracy: 93.8%
Validation accuracy: 78.8%
Minibatch loss at step 350: 0.307479
Minibatch accuracy: 93.8%
Validation accuracy: 77.6%
Minibatch loss at step 400: 1.048190
Minibatch accuracy: 62.5%
Validation accuracy: 79.2%
Minibatch loss at step 450: 0.456180
Minibatch accuracy: 87.5%
Validation accuracy: 80.9%
Minibatch loss at step 500: 0.726122
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [6]:
batch_size = 128

graph = tf.Graph()

with graph.as_default():

  # Input data.
  X = tf.placeholder(
    tf.float32, shape=(None, image_size, image_size, num_channels))
  Y = tf.placeholder(tf.float32, shape=(None, num_labels))
  
  # Variables.
  w1 = tf.get_variable('w1', shape=[3, 3, 1, 32], initializer=tf.contrib.layers.xavier_initializer())
  b1 = tf.Variable(tf.zeros([32]))
  w2 = tf.get_variable('w2', shape=[3, 3, 32, 64], initializer=tf.contrib.layers.xavier_initializer())
  b2 = tf.Variable(tf.zeros([64]))
  w3 = tf.get_variable('w3', shape=[12 * 12 * 64, 128], initializer=tf.contrib.layers.xavier_initializer())
  b3 = tf.Variable(tf.zeros([128]))
  w4 = tf.get_variable('w4', shape=[128, 10], initializer=tf.contrib.layers.xavier_initializer())
  b4 = tf.Variable(tf.zeros([10]))
  keep_prob = tf.placeholder(tf.float32)
  
  # Model.
  def model(data, keep_prob):
    conv = tf.nn.conv2d(data, w1, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + b1)
    conv = tf.nn.conv2d(hidden, w2, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + b2)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    drop = tf.nn.dropout(pool, keep_prob)
    shape = drop.get_shape().as_list()
    reshape = tf.reshape(pool, [-1, shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, w3) + b3)
    drop = tf.nn.dropout(hidden, keep_prob)
    return tf.matmul(drop, w4) + b4
  
  # Training computation.
  logits = model(X, keep_prob)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)
  starter_learning_rate = 0.001
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           10000, 0.96, staircase=True)
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
num_steps = 15000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset_val = (step * batch_size) % (valid_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    batch_data_valid = valid_dataset[offset_val:(offset_val + batch_size), :, :, :]
    batch_labels_valid = valid_labels[offset_val:(offset_val + batch_size), :]
    feed_dict = {X: batch_data, Y: batch_labels, keep_prob: 0.5}
    _, l, train_accuracy = session.run(
      [optimizer, loss, accuracy], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % (train_accuracy * 100))
      val_accuracy = accuracy.eval(feed_dict={X: batch_data_valid, Y: batch_labels_valid, keep_prob: 1.})
      print('Validation accuracy: %.1f%%' % (val_accuracy * 100))
        
  # test set to big to be evaluated at once so it must be split
  test_dataset_split, test_labels_split = np.split(test_dataset, 10), np.split(test_labels, 10)
  test_accuracy = 0
  for i in xrange(10):
    test_accuracy += accuracy.eval(feed_dict={X: test_dataset[i], Y: test_labels[i], keep_prob: 1.})
  test_accuracy /= 10
  print('Test accuracy: %.1f%%' % (test_accuracy * 100))

Initialized
Minibatch loss at step 0: 2.319197
Minibatch accuracy: 8.6%
Validation accuracy: 9.4%
Minibatch loss at step 500: 0.419884
Minibatch accuracy: 83.6%
Validation accuracy: 85.9%
Minibatch loss at step 1000: 0.475233
Minibatch accuracy: 84.4%
Validation accuracy: 89.8%
Minibatch loss at step 1500: 0.448498
Minibatch accuracy: 85.9%
Validation accuracy: 91.4%
Minibatch loss at step 2000: 0.439836
Minibatch accuracy: 90.6%
Validation accuracy: 85.9%
Minibatch loss at step 2500: 0.388376
Minibatch accuracy: 89.1%
Validation accuracy: 89.8%
Minibatch loss at step 3000: 0.474728
Minibatch accuracy: 85.2%
Validation accuracy: 93.8%
Minibatch loss at step 3500: 0.394643
Minibatch accuracy: 89.1%
Validation accuracy: 95.3%
Minibatch loss at step 4000: 0.362668
Minibatch accuracy: 87.5%
Validation accuracy: 95.3%
Minibatch loss at step 4500: 0.258134
Minibatch accuracy: 92.2%
Validation accuracy: 91.4%
Minibatch loss at step 5000: 0.316939
Minibatch accuracy: 89.8%
Validation accuracy: